## 1.引言
在[上一节](https://golfxiao.blog.csdn.net/article/details/144999291) 我们着重介绍了指令微调的批量数据构造过程，这一节我们将在上一节的基础上，接着介绍如何实际进行指令微调训练。

对比前面的预训练和分类微调不难发现，模型的训练步骤基本上大同小异，不论是进行预训练还是SFT，都是采用小批量梯度下降，并且混合精度训练、分布式训练这些训练优化方法对预训练和SFT都适用。到目前为止，唯一有区别的两个地方在于批量数据收集的方式和注意力掩码。所以我们就不再为SFT编写单独的训练器，而是基于前面预训练阶段封装过的训练器进行局部改造，以支持指令微调训练。

## 2. 训练器改造

首先，加载在预训练阶段封装好的训练器代码。

In [1]:
%run trainer.py

#### 2.1 支持collator收集器

SFT相比于预训练多了一个批量数据处理，我们可以在设置数据集时扩展一个参数来支持`批量数据收集器`的配置。

In [16]:
def set_dataset(self, train_set, eval_set, batch_collator=None):
    self.train_set = train_set
    self.eval_set = eval_set
    self.batch_collator = batch_collator
    print(f'set trainset: {len(train_set)}, evalset: {len(eval_set)}') if self.verbose else None

setattr(Trainer, "set_dataset", set_dataset)

同时，还需要在创建DataLoader时，添加批量数据收集器的设置，通过`collate_fn`来指定。

In [17]:
def _init_dataloader(self):
    assert self.train_set and self.eval_set, f"train_set and eval_set can't be empty."
    train_set, eval_set, batch_size = self.train_set, self.eval_set, self.batch_size
    sampler = DistributedSampler(train_set) if self.ddp else None
    self.train_loader = DataLoader(train_set, 
                                   batch_size=batch_size, 
                                   shuffle=(sampler==None), 
                                   num_workers=0, 
                                   drop_last=True, 
                                   collate_fn=self.batch_collator,
                                   sampler=sampler)
    self.eval_loader = DataLoader(eval_set, 
                                  batch_size=batch_size, 
                                  shuffle=True, 
                                  num_workers=0, 
                                  drop_last=False,
                                  collate_fn=self.batch_collator)
    self.steps_per_epoch = len(self.train_loader)
    self.total_steps = self.num_epochs * self.steps_per_epoch

setattr(Trainer, "_init_dataloader", _init_dataloader)

#### 2.2 引入注意力掩码
在之前对SFT数据的准备时，会因为每批数据长度不等而对数据作填充对齐，因此需要引入注意力掩码attention_mask来取消对padding token的注意力计算，attention_mask的作用在于指示哪些位置是有效的（模型应该关注），哪些位置是填充的（模型应该忽视）。

具体到使用位置，attention_mask应该是在计算出缩放点积的注意力得分之后来运用，通常是 softmax 之前的得分矩阵，这样不会影响归一化。

为了方便演示，我们随机初始化一个形状为(2,2,3,3)的注意力分数，并对应的创建一个形状为（2,3）的注意力掩码。我们将以这个随机的数据为例，来讨论注意力掩码如何作用于注意力分数的计算。

> 注：(2,2,3,3)中的4个数字分别表示批量大小batch_size、多头数量num_heads、序列长度num_tokens、序列长度num_tokens。

In [12]:
import torch  

scaled_atten_scores = torch.randn(2, 2, 3, 3)  # 随机生成的注意力分数  
# 注意力掩码，1表示有效，0表示填充  
attention_mask = torch.tensor([[1, 0, 1],  # 第一个样本，第二个词是填充  
                                [0, 1, 1]]) # 第二个样本，第一个词是填充  
# 输出示例数据  
print("scaled_atten_scores:\n", scaled_atten_scores)  
print("attention_mask:\n", attention_mask)


scaled_atten_scores:
 tensor([[[[-1.0607, -2.3244,  1.2311],
          [-1.0973, -0.9669,  0.6712],
          [-0.9405, -0.4681,  0.6455]],

         [[-0.8957,  1.1124, -0.4168],
          [-1.7106, -0.3290,  1.3966],
          [-0.9949, -1.2096, -0.5560]]],


        [[[-2.7202,  0.5421,  0.6557],
          [-1.4056, -1.2743,  0.4513],
          [-0.2280,  0.9224,  0.2056]],

         [[-0.4970,  1.2782,  2.5501],
          [-0.3018, -0.6703, -0.6171],
          [-0.8334,  0.4839, -0.1349]]]])
attention_mask:
 tensor([[1, 0, 1],
        [0, 1, 1]])


由于`attention_mask`与`scaled_atten_scores`的张量形状不同，无法直接进行运算，因此需要将`attention_mask`扩展成 (batch_size, num_heads, num_tokens, num_tokens)，这样它可以与多头注意力得分矩阵形状匹配。
> unsqueeze:用于调整张量的维度，它可以在指定位置插入一个维度，新插入维度的默认大小为1。
> expand: 扩展张量大小，使其在指定维度上重复。这两个方法结合，可以方便的调整张量形状以满足特定的形状。

In [14]:
# 扩展 attention_mask 的维度  
batch_size, num_heads, num_tokens, _ = scaled_atten_scores.shape
# Expand attention_mask to (batch_size, 1, num_tokens, num_tokens)
expanded_mask = attention_mask.unsqueeze(1).unsqueeze(2)

# Make sure mask is broadcastable to attention_scores shape
# expanded_mask = expanded_mask.expand(batch_size, num_heads, num_tokens, num_tokens)
expanded_mask, expanded_mask.shape

(tensor([[[[1, 0, 1]]],
 
 
         [[[0, 1, 1]]]]),
 torch.Size([2, 1, 1, 3]))

将扩维后的掩码转换为布尔张量，可以方便的表示哪些位置是填充的，如果是填充(0)则为True，不是填充（1）则为False。

In [17]:
mask = expanded_mask == 0
mask

tensor([[[[False,  True, False]]],


        [[[ True, False, False]]]])

将扩维后的 attention_mask 应用于注意力得分矩阵。
> 我们在使用masked_fill方法时，它会自动将mask的形状调整成与scaled_atten_scores的形状，在我们这个例子中，它会自动从`[2, 1, 1, 3]`调整为`[2, 2, 3, 3]`,这在张量操作中被称为广播机制。

In [18]:
masked_scaled_atten_scores = scaled_atten_scores.masked_fill(mask, -torch.inf) 
masked_scaled_atten_scores

tensor([[[[-1.3703,    -inf,  2.3785],
          [-0.8002,    -inf,  0.1630],
          [-1.1414,    -inf, -0.5628]],

         [[ 0.0068,    -inf,  0.1025],
          [-0.5501,    -inf, -0.4838],
          [-0.9264,    -inf,  1.7862]]],


        [[[   -inf,  1.8924, -0.9420],
          [   -inf,  1.5154, -0.3482],
          [   -inf,  0.8034, -0.8956]],

         [[   -inf,  0.4343, -0.6637],
          [   -inf, -0.0628, -0.3497],
          [   -inf,  1.1594, -0.6931]]]])

masked_fill方法可以将attention_mask中值为True的位置填充为-torch.inf（负无穷大）。这样在后续的softmax计算中，这些填充位置的概率值就趋近于0.

In [19]:
atten_weights = torch.softmax(masked_scaled_atten_scores, dim=-1)
atten_weights

tensor([[[[0.0230, 0.0000, 0.9770],
          [0.2763, 0.0000, 0.7237],
          [0.3593, 0.0000, 0.6407]],

         [[0.4761, 0.0000, 0.5239],
          [0.4834, 0.0000, 0.5166],
          [0.0622, 0.0000, 0.9378]]],


        [[[0.0000, 0.9445, 0.0555],
          [0.0000, 0.8657, 0.1343],
          [0.0000, 0.8454, 0.1546]],

         [[0.0000, 0.7499, 0.2501],
          [0.0000, 0.5712, 0.4288],
          [0.0000, 0.8644, 0.1356]]]])

相对应的在训练器中也需要对attention_mask作支持。但这个代码的修改会比较碎，需要从train_epoch->train_step/evaluate->model.forward->decode_layer.forward->attn.forward整个方法调用链都需要扩展参数，所以这里只作个示意，具体可以自行修改。

In [ ]:
def _train_epoch(self, cur_epoch):
    ……
    for i, (X, Y, attnmask) in enumerate(self.train_loader):  
        ……
        train_loss = self._train_step(X.to(self.device), Y.to(self.device), attnmask.to(self.device))
        ……

## 3. 指令训练

#### 3.1 加载预训练模型
首先，加载之前封装的模型结构代码，以及和[上一节](https://golfxiao.blog.csdn.net/article/details/144999291)封装的指令微调数据处理代码。

In [1]:
%run sft_dataset.py
%run transformer.py
%run trainer.py

创建模型实例并加载预训练的模型权重参数。
> 注：由于SFT与预训练的训练方法不同，所以这里我们只使用预训练阶段的模型状态`model_state`，不使用优化器状态`optimizer_state`和训练进度信息`epoch`、`step`。

In [2]:
device = 'cuda:2'
config = GPTConfig(flash_attn=False)
model = MiniGPT(config).to(device)
pretrain_checkpoint_path = "/data2/minigpt/models/20241210/checkpoint-450000.pth"
checkpoint = torch.load(pretrain_checkpoint_path, map_location=device, weights_only=False)
model.load_state_dict(checkpoint['model_state'])
model.token_emb.weight.data

tensor([[ 0.1634,  1.0951, -0.2334,  ..., -0.6627,  1.5544, -0.4751],
        [ 0.0926,  1.1536, -0.6867,  ..., -0.3443,  1.1320, -0.4675],
        [ 0.0376,  0.9540,  0.0747,  ...,  0.4608, -1.1311,  1.3976],
        ...,
        [ 0.4062,  0.3360,  0.1567,  ...,  0.8342, -0.1043, -0.9669],
        [ 0.4914, -1.2539,  0.5454,  ...,  1.4870, -0.4196, -0.0348],
        [-0.0737, -0.1939,  0.1602,  ..., -0.8079,  0.3017,  0.5944]],
       device='cuda:2')

加载之前训练好的分词器，共有32000词汇。

In [3]:
tokenizer_path = "/data2/minigpt/models/tokenizer_v3"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)
len(tokenizer)

32000

编写一个指令遵循测试方法，并用它来测试微调之前模型的指令遵循能力。

In [10]:
def instruction_test(model, tokenizer, text, device):
    messages = [{"role": "user", "content": text}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = torch.tensor([tokenizer.encode(prompt)]).to(device)
    response_ids = model.generate(inputs, 512, tokenizer.eos_token_id)
    return tokenizer.decode(response_ids.squeeze(0), skip_special_tokens=False)

In [6]:
instruction = "如果我需要一个适合智能家居的中央处理器，你有什么推荐吗？"
response = instruction_test(model, tokenizer, instruction, device)
print(response)

<|im_start|>user
如果我需要一个适合智能家居的中央处理器，你有什么推荐吗？<|im_end|>
<|im_start|>assistant
我们以客户为本，以客户为中心，以客户满意为宗旨，以客户满意为目标，以服务为准则，以客户满意为目标，以客户满意为目标，以服务为准则，以客户满意为目标，以服务为准则，以服务为准则，以客户满意为目标，以客户满意为目标，以服务为准则，以服务为准则，以客户满意为目标，以服务为准则，以客户满意为目标，以服务为准则，以客户满意为目标，以服务为准则，以服务为准则，以服务为准则，以客户满意为目标，以服务为准则，以客户满意为目标，以服务为准则，以客户满意为目标，以服务为准则，以客户满意为目标，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以客户满意为目标，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以服务为准则，以


> 可以看到，此时模型生成的文本与我们给出的用户指令`如果我需要一个适合智能家居的中央处理器，你有什么推荐吗？`没有丝毫联系，模型还不知道什么是指令以及如何遵循指令，这也正是我们可以通过SFT来优化的地方。

#### 3.2 微调训练

首先，我们加载指令微调数据集，并按照指定比例`0.96`和`0.02`分割。
> 注：在这个分割比例中，我们将大部分数据都留给了训练阶段（训练占96%，验证占2%），目的是避免验证阶段耗时太长。与之前预训练阶段唯一不同的是，多分了一个测试集`test`(也占2%)出来，用于微调完的评估测试。

In [4]:
train_ratio, eval_ratio = 0.96, 0.02
dataset_path = "/data2/minigpt/dataset/sft/sft_data_zh.jsonl"
ds = InstructionDataset(dataset_path, tokenizer, config.context_length, max_lines=100000)
train_set, eval_set, test_set = split_dataset(ds, train_ratio, eval_ratio)
len(train_set), len(eval_set), len(test_set)

calculate lines[100000] use time: 0.000s


(96000, 2000, 2000)

配置训练参数，并创建参数优化器`optimizer`（这与预训练阶段相同）。

In [5]:
last_checkpoint_path = ""
output_dir = "/data2/minigpt/models/sft/20250113"
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

train_args = {
    "train_batch_size": 8,
    "eval_strategy": "step",
    "eval_steps": 1000,
    "warmup_steps": 1000,
    "save_strategy": "step",
    "save_steps": 10000,
    "num_train_epochs": 2,
    "output_dir": output_dir,
    "last_checkpoint_path": last_checkpoint_path,
    "use_mixed_precision": True,
}

创建训练器，先为训练器设置随机数种子，并设置训练集、验证集以及批量数据收集器`batch_collator`，随后调用`trainer.train`开始训练。

In [7]:
start_time = time.time()
trainer = Trainer(model, optimizer, train_args, device, verbose=True)
trainer.set_seed(123)
trainer.set_dataset(train_set, eval_set, create_batch_collator(tokenizer))
trainer.train()
print(f"train use time: {(time.time()-start_time)/60:.2f}min") if trainer.verbose else None

set seed to 123
set trainset: 96000, evalset: 2000
init train_loader steps: 12000, eval_loader: 250
init grad scaler:  True
2025-01-13 10:46:38 start epoch:0 from step:0
2025-01-13 10:50:30 lr=0.00010, train_loss: 3.2183, eval_loss: 2.9650, grad_norm=1.00000, steps: 999/24000
2025-01-13 10:53:19 lr=0.00010, train_loss: 3.0105, eval_loss: 2.8519, grad_norm=1.00000, steps: 1999/24000
2025-01-13 10:56:10 lr=0.00010, train_loss: 2.9181, eval_loss: 2.7942, grad_norm=1.00000, steps: 2999/24000
2025-01-13 10:59:02 lr=0.00010, train_loss: 2.8671, eval_loss: 2.7622, grad_norm=1.00000, steps: 3999/24000
2025-01-13 11:01:51 lr=0.00009, train_loss: 2.8309, eval_loss: 2.7177, grad_norm=1.00000, steps: 4999/24000
2025-01-13 11:04:42 lr=0.00009, train_loss: 2.7947, eval_loss: 2.6954, grad_norm=1.00000, steps: 5999/24000
2025-01-13 11:07:30 lr=0.00009, train_loss: 2.7606, eval_loss: 2.6673, grad_norm=1.00000, steps: 6999/24000
2025-01-13 11:10:16 lr=0.00008, train_loss: 2.7483, eval_loss: 2.6504, grad

通过这10万条数据的训练，模型的训练损失从3.21下降到了2.41，验证损失从2.96下降到了2.48。并且从数据趋势来看，这两个损失还在持续下降过程中，理论上这个训练如果多跑几轮，模型的损失应该还能继续降低。

#### 3.3 评估测试

我们编写一个评估测试方法`test`，并使用训练阶段从未见过的测试数据集`test_set`对模型损失进行评估，以验证训练结果是否具有泛化能力。

In [8]:
def test(model, dataloader, device):
    model.eval()
    num_batches = len(dataloader)
    total_loss = 0
    
    for batch in dataloader:
        X, Y = batch[0].to(device), batch[1].to(device) 
        with torch.no_grad():
            logits = model(X)
        loss = f.cross_entropy(logits.flatten(0, 1), Y.flatten())
        total_loss += loss.item()
    
    model.train()
    return total_loss/num_batches 

testloader = DataLoader(test_set, batch_size=trainer.batch_size, collate_fn=trainer.batch_collator)
loss = test(trainer.model, testloader, device)
loss

2.4950199112892153

可以看到，模型在从未见过的测试集上平均损失为`2.4950`，模型在训练阶段的最低验证损失为`2.4813`，两者差异并不大，说明上面的训练从损失指标上来看是有效的。

接下来，我们再使用上面封装的方法`instruction_test`，来测试指令微调后的效果变化。

In [11]:
instruction = "如果我需要一个适合智能家居的中央处理器，你有什么推荐吗？"
response = instruction_test(trainer.model, tokenizer, instruction, device)
print(response)

<|im_start|>user
如果我需要一个适合智能家居的中央处理器，你有什么推荐吗？<|im_end|>
<|im_start|>assistant
当然，我可以为你推荐一些适合智能家居的中央处理器。
1. 英特尔酷睿i5：英特尔酷睿i5是一款高性能的处理器，支持4K HDR屏幕，支持4K视频录制，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR屏幕显示，支持4K HDR


从模型生成的第一句话`当然，我可以为你推荐一些适合智能家居的中央处理器。`来看，模型已经知道要按照`user`给出的指令`如果我需要一个适合智能家居的中央处理器，你有什么推荐吗？`来回答问题。这说明，经过上面10万条数据的2轮训练，模型已经初步学会了遵循指令。

#### 3.4 增大训练数据量
上面的结果中虽然一开始遵循了指令，但后面就在不停的重复前面的内容（如`支持4K HDR屏幕显示`)，这是因为我们训练时只使用了一个10万条的小数据集。要想改善生成文本的效果，就需要加大训练数据量，以及增加机器资源。

我这边使用4张24G显卡对整个SFT数据集进行了完整的一轮训练，大概用时30多个小时，具体训练过程就不展开，我们只在这里演示里训练结果。
> 注：下面的`checkpoint-330000.pth`是我们一轮训练后的checkpoint，我们基于此checkpoint进行指令遵循的测试。

In [17]:
model2 = MiniGPT(config).to(device)
sft_checkpoint_path = "/data2/minigpt/models/sft/20241020/checkpoint-330000.pth"
sft_checkpoint = torch.load(sft_checkpoint_path, map_location=device, weights_only=False)
model2.load_state_dict(sft_checkpoint['model_state'])
response = instruction_test(model2, tokenizer, instruction, device)
print(response)

<|im_start|>user
如果我需要一个适合智能家居的中央处理器，你有什么推荐吗？<|im_end|>
<|im_start|>assistant
当您需要一个适合智能家居的中央处理器时，以下是一些推荐：

1. 联邦处理器（Federated CPU）：这种处理器通常比其他处理器更轻，更灵活，并且具有更高的安全性和性能。

2. 固态硬盘（SSD）：SSD是一种固态硬盘，它比传统的机械硬盘更轻便，更易于存储和读取。

3. 固态硬盘（SSD）：SSD是一种固态硬盘，它比机械硬盘更轻，更易于存储和读取。

4. 光驱（Federated CPU）：光驱是一种机械设备，它通常比机械设备更轻，更灵活，并且具有更高的安全性和性能。

5. 磁盘（HDD）：磁盘是一种存储设备，它通常比机械设备更轻，更易于存储和读取。

6. 内存（RAM）：内存是一种存储器，它比机械设备更轻，更灵活，并且具有更高的安全性和性能。

7. 电源（Power）：Power是一种电源，它比机械设备更轻，更易于存储和读取。

8. 声卡（Voice）：声卡是一种声卡，它比机械设备更轻，更灵活，并且具有更高的安全性和性能。

9. 扬声器（Light）：扬声器是一种扬声器，它比机械设备更轻，更灵活，并且具有更高的安全性和性能。

10. 磁盘（Memory）：磁盘是一种存储设备，它比机械设备更轻，更灵活，并且具有更高的安全性和性能。<|im_end|>


> 可以看到，相同的用户指令，在上面这个经过更充分训练的模型回答结果中，回答的句子已经比较完整，前面的循环重复问题已经基本消失。

**小结**：本文在上一篇的基础上，深入探讨了指令微调训练的具体实施过程。首先，我们对先前预训练阶段所封装的Trainer进行了改造，以便支持自定义的批量数据处理器。接着，我们对注意力计算流程进行了优化，增加了对注意力损失掩码的支持。最后，使用一个大小为10万条的数据集，演示了模型的指令微调训练过程，同时也测试了经过33万步训练后的模型的文本生成效果。我们的训练仍然不够充分，你如果有条件，可以训练更多的轮数，以及尝试更多不同种类的数据集。